In [2]:
%%capture
!pip install scanpy
!pip install KDEpy
!pip install leidenalg

In [19]:
%%capture
!pip install -e ../../tools/nomad/

In [1]:
import os
import sys
from datetime import datetime

import pandas as pd
import scanpy as sc
import anndata as ad
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sps
from scipy.stats import spearmanr

# to be able to import from tools dir
module_path = os.path.abspath(os.path.join("../../"))
if module_path not in sys.path:
    sys.path.append(module_path)

import tools.util_probe as up
import tools.util as ut
import tools.NB_est as nb
import tools.countsplit as cs
import tools.scDEED as scd
import tools.clustering_opt as co
import tools.util_plot as nmd_plot
import tools.util_imputation as nmd_imp

import fi_nomad as nmd
from fi_nomad.types import kernelInputTypes
from fi_nomad.types import KernelStrategy
from fi_nomad.types import InitializationStrategy

import warnings

warnings.filterwarnings("ignore")

from scipy.stats import kendalltau
import importlib
import logging

In [2]:
# logging has to manually turned on to see nomad output
logging.basicConfig(level=logging.INFO)

## Read data

In [3]:
data_path = "../../data/S2"
figure_path = f"{data_path}/figures/imputation"
layer = "counts"

In [4]:
data_counts = sc.read_h5ad(data_path + "/filtered_data_maxpool_processed_and_pca.h5ad")
data_counts

AnnData object with n_obs × n_vars = 1255 × 5540
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'outlier', 'n_genes', 'total_counts_norm', 'total_counts_scale', 'embedding_reliability', 'reliability_score', 'null_reliability_score', 'leiden_opt_PCA'
    var: 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'var_counts', 'is_scd_outlier', 'nb_overdisp', 'nb_overdisp_cutoff', 'nb_mean', 'nb_umi', 'Intercept_step1_sct', 'log_umi_step1_sct', 'dispersion_step1_sct', 'mean', 'std'
    uns: 'BacSC_params_PCA', 'PCA', 'leiden', 'leiden_opt_PCA_colors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts', 'normalized_counts', 'vst_counts'
    obsp: 'PCA_connectivities', 'PCA_distances'

Read expression level class data

In [5]:
expr_class = pd.read_csv(
    "../../data/control_expression_classification.tsv", delimiter="\t"
)
expr_class.set_index(keys="locus_tag", inplace=True)
expr_class

,Name,tpm,expression_level,log_tpm
locus_tag,,,,
PA0001,dnaA,333.097801,high,8.379802
PA0002,dnaN,299.127321,high,8.224616
PA0003,recF,128.151975,medium_high,7.001712
PA0004,gyrB,138.579052,medium_high,7.114565
PA0005,lptA,141.413217,medium_high,7.143773
...,...,...,...,...
PA5566,PA5566,45.077831,medium_low,5.494346
PA5567,PA5567,49.149167,medium_low,5.619095
PA5568,PA5568,208.178186,high,7.701675


- `mean_counts`: mean gene expression
- `n_cells`: number of cells that express gene (so (n - `n_cells`) / n = sparsity)

In [6]:
data_counts.var = data_counts.var.join(expr_class)
data_counts.var

,feature_types,genome,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,n_cells,var_counts,...,nb_umi,Intercept_step1_sct,log_umi_step1_sct,dispersion_step1_sct,mean,std,Name,tpm,expression_level,log_tpm
PA0001,Gene Expression,PA01,508,0.529880,0.425190,59.521912,665.0,6.501290,508,0.555083,...,2.142832,-6.610836,2.011342,0.009302,1.105347,0.166192,dnaA,333.097801,high,8.379802
PA0002,Gene Expression,PA01,383,0.399203,0.335903,69.482072,501.0,6.218600,383,0.491633,...,2.263982,NaN,NaN,NaN,0.770695,0.190039,dnaN,299.127321,high,8.224616
PA0003,Gene Expression,PA01,604,0.728287,0.547131,51.872510,914.0,6.818924,604,1.023383,...,2.035867,NaN,NaN,NaN,1.244879,0.176504,recF,128.151975,medium_high,7.001712
PA0004,Gene Expression,PA01,310,0.311554,0.271213,75.298805,391.0,5.971262,310,0.372257,...,2.366935,NaN,NaN,NaN,0.465868,0.219582,gyrB,138.579052,medium_high,7.114565
PA0005,Gene Expression,PA01,108,0.103586,0.098565,91.394422,130.0,4.875197,108,0.131103,...,2.733139,NaN,NaN,NaN,-1.030973,0.314385,lptA,141.413217,medium_high,7.143773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PA5566,Gene Expression,PA01,6,0.004781,0.004769,99.521912,6.0,1.945910,6,0.004758,...,3.211789,-15.931712,3.485019,0.010852,-4.151651,0.232698,PA5566,45.077831,medium_low,5.494346
PA5567,Gene Expression,PA01,127,0.117928,0.111477,89.880478,148.0,5.003946,127,0.137487,...,2.692861,NaN,NaN,NaN,-0.840724,0.317603,PA5567,49.149167,medium_low,5.619095
PA5568,Gene Expression,PA01,466,0.506773,0.409970,62.868526,636.0,6.456770,466,0.598958,...,2.168977,NaN,NaN,NaN,1.040103,0.171432,PA5568,208.178186,high,7.701675
PA5569,Gene Expression,PA01,1104,2.755378,1.323189,12.031873,3458.0,8.148735,1104,8.395140,...,1.816896,-5.261660,2.109112,0.094811,1.075991,0.444127,rnpA,3387.343294,high,11.725938


## Fit NMD

In [7]:
latent_rank = 10
momentum_beta = 0.7
max_iterations = 1000

init_strat = InitializationStrategy.ROWWISE_MEAN
kernel_strat = KernelStrategy.MOMENTUM_3_BLOCK_MODEL_FREE

In [8]:
m, n = data_counts.X.shape
X_nmd = ut.convert_to_dense(data_counts, layer=layer)
X_nmd = X_nmd.astype(float)

In [9]:
result = nmd.decompose(
    X_nmd,
    latent_rank,
    kernel_strategy=kernel_strat,
    initialization=init_strat,
    kernel_params=kernelInputTypes.Momentum3BlockAdditionalParameters(
        momentum_beta=momentum_beta
    ),
    manual_max_iterations=max_iterations,
    verbose=True,
)

INFO:fi_nomad.entry:	Initiating run, target_rank: 10, tolerance: None
INFO:fi_nomad.entry:1000 total, final loss Not Tracked
INFO:fi_nomad.entry:	Initialization took 19.044169648026582 loop took 185.3114944039844 overall (0.1853114944039844/ea)


In [10]:
data_counts.obsm["X_nmd"] = result.factors[0]
data_counts.varm["NMD_components"] = result.factors[1].T
data_counts.layers["Theta"] = result.factors[0] @ result.factors[1]

Output file for pretty plotting in R

In [11]:
pd.DataFrame(data_counts.layers["Theta"]).to_csv(
    f"{data_path}/Theta_3BNMD_r{latent_rank}.csv", index=False
)

## Sample gene pairs

In [14]:
importlib.reload(nmd_imp)

<module 'tools.util_imputation' from '/Users/stffn/projects/thesis_dev/msc_thesis/application/tools/util_imputation.py'>

In [15]:
n_genes = 50  # number of gene pairs
expression_levels_2_compare = ["low", "medium_high"]

In [16]:
gene_pairs = nmd_imp.sample_gene_pairs(
    data_counts,
    n=n_genes,
    measure_var=["mean_counts", "pct_dropout_by_counts"],
    expr_lvls=expression_levels_2_compare,
)
gene_pairs.head()

,low,medium_high,dist,gene_pair_id
0,PA0570,PA1139,0.000026,0
1,PA2018,PA1651,0.000000,1
2,PA2599,PA2489,0.000000,2
3,PA0825,PA4108,0.000000,3
4,PA2552,PA1915,0.000000,4


Get all negative Theta values.

In [17]:
gene_pair_df = nmd_imp.make_gene_pair_df(gene_pairs, adata=data_counts)
gene_pair_df.sort_values(by=["gene_pair_id", "dist"]).head()

,Gene,Expression,gene_pair_id,dist,expression_lvl_class
0,PA0570,0.043325,0,0.000026,low
1,PA0570,0.065547,0,0.000026,low
2,PA0570,0.039778,0,0.000026,low
3,PA0570,0.019333,0,0.000026,low
4,PA0570,0.020767,0,0.000026,low


## Export to csv for visualization in R

In [18]:
gene_pair_df.to_csv(
    f"{data_path}/gene_pair_df_4_viz_{n_genes}genes_3BNMD_rank{latent_rank}_{expression_levels_2_compare[0]}-{expression_levels_2_compare[1]}_mean_and_sparsity_whereXzero.csv",
    index=False,
)